### INSURANCE POLICY LAPSE PREDICTION

### Import Packages

In [1]:
#import basic packages.
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier

### Import Files

In [2]:
#import files
client_df = pd.read_csv('Practice/data/client_data.csv')
payment_df = pd.read_csv('Practice/data/payment_history.csv')
policy_df = pd.read_csv('Practice/data/policy_data.csv')
main_train_df = pd.read_csv('Practice/data/train.csv')
smp_sub_df = pd.read_csv('Practice/data/sample_sub.csv')

### Expolaratory Data Analysis

In [3]:
#EDA on main_train_df

main_train_df.head(3)

,Policy ID,Lapse,Lapse Year
0,PID_4928TWH,?,?
1,PID_KBLLEGK,?,?
2,PID_90F0QA3,?,?


The Policy ID with Lapse and Lapse Year '?' values are meant to be in the test df and will be separated

In [4]:
main_train_df.shape

(51685, 3)

The train df has 51,683 rows and 3 columns

In [5]:
main_train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51685 entries, 0 to 51684
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Policy ID   51685 non-null  object
 1   Lapse       51685 non-null  object
 2   Lapse Year  51685 non-null  object
dtypes: object(3)
memory usage: 1.2+ MB


In [6]:
main_train_df.isna().sum()

Policy ID     0
Lapse         0
Lapse Year    0
dtype: int64

No null values

In [7]:
len(main_train_df['Policy ID'].unique())

51685

There are no duplicate Policy ID values

In [8]:
main_train_df['Lapse'].unique()

array(['?', '1'], dtype=object)

Apart from '?', the Lapse value is equal to 1

In [9]:
main_train_df['Lapse Year'].unique()

array(['?', '2019', '2018', '2017'], dtype=object)

The Lapse Years are between (2017-2019)

### Feature Engineering

First, we create the test df. Our target variable is Lapse, therefore we drop the Lapse Year column

In [10]:
test_df = main_train_df[main_train_df['Lapse'] == '?'].drop(['Lapse Year'], axis =1)
test_df.head(3)

,Policy ID,Lapse
0,PID_4928TWH,?
1,PID_KBLLEGK,?
2,PID_90F0QA3,?


In [11]:
test_df['Lapse'].unique()

array(['?'], dtype=object)

In [12]:
test_df.shape

(43707, 2)

Out of the 51,685 rows, 43,707 are in our test df

We create the train_df without the test rows

In [13]:
train_df = main_train_df[main_train_df['Lapse'] != '?']
train_df.head(3)

,Policy ID,Lapse,Lapse Year
16,PID_MFAAYNJ,1,2019
23,PID_TICDPAY,1,2019
30,PID_SPACC3N,1,2018


In [14]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7978 entries, 16 to 51680
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Policy ID   7978 non-null   object
 1   Lapse       7978 non-null   object
 2   Lapse Year  7978 non-null   object
dtypes: object(3)
memory usage: 249.3+ KB


Our train df has 7,978 columns. The Lapse and Lapse Year are stored as objects. We will convert them to integers

In [15]:
train_df.loc[:, ('Lapse Year')] = pd.to_numeric(train_df['Lapse Year'])
train_df.loc[:, ('Lapse')] = pd.to_numeric(train_df['Lapse'])
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7978 entries, 16 to 51680
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Policy ID   7978 non-null   object
 1   Lapse       7978 non-null   int64 
 2   Lapse Year  7978 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 249.3+ KB


C:\Users\USER\SeraApps\anaconda\lib\site-packages\pandas\core\indexing.py:1843: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value
C:\Users\USER\SeraApps\anaconda\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


Success!

In [16]:
train_df['Lapse'].unique()

array([1], dtype=int64)

In [17]:
train_df['Lapse Year'].unique()

array([2019, 2018, 2017], dtype=int64)